# Backpropagation from scratch

Fully matrix-based approach to backpropagation over a mini-batch. Trained and tested on the iris data

In [43]:
import random
import numpy as np
#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y , x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a))
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch2(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        X = [x for x, y in mini_batch]
        Y = [y for x, y in mini_batch]
        X = np.concatenate(X, axis=1)
        Y = np.concatenate(Y, axis=1)
        
        nabla_b, nabla_w = self.backprop(X, Y)

        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # feedforward
        activation = np.concatenate((x, np.ones((1, x.shape[1]))), axis=0)
        activations = [activation] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for _d, _a in zip(delta.transpose(), activations[-2].transpose()):
            _d = np.reshape(_d, [len(_d), 1])
            _a = np.reshape(_a, [len(_a), 1])
            nabla_w[-1] += np.dot(_d, _a.transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            for _d, _a in zip(delta.transpose(), activations[-l-1].transpose()):
                _d = np.reshape(_d, [len(_d), 1])
                _a = np.reshape(_a, [len(_a), 1])
                nabla_w[-l] += np.dot(_d, _a.transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)
    
    def feedforward2(self, a):
        zs = []
        activations = [a]

        activation = a
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) 
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)

        return (zs, activations)

    def update_mini_batch2(self, mini_batch, eta):
        batch_size = len(mini_batch)

        # transform to (input x batch_size) matrix
        X = np.asarray([_x.ravel() for _x, _y in mini_batch]).transpose()
        # transform to (output x batch_size) matrix
        Y = np.asarray([_y.ravel() for _x, _y in mini_batch]).transpose()

        nabla_b, nabla_w = self.backprop2(X, Y)
        self.weights = [w - (eta / batch_size) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / batch_size) * nb for b, nb in zip(self.biases, nabla_b)]

        return

    def backprop2(self, x, y):

        nabla_b = [0 for i in self.biases]
        nabla_w = [0 for i in self.weights]

        # feedforward
        zs, activations = self.feedforward2(x)

        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta.sum(1).reshape([len(delta), 1]) # reshape to (n x 1) matrix
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta.sum(1).reshape([len(delta), 1]) # reshape to (n x 1) matrix
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())

        return (nabla_b, nabla_w)




In [44]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import pandas as pd

# Load the iris dataset
iris = load_iris()

# Shuffle the dataset
indices = np.arange(iris.data.shape[0])
np.random.shuffle(indices)

# Split the dataset into a training set and a test set
train_size = int(0.8 * len(indices))  # 80% of data for training
train_indices = indices[:train_size]
test_indices = indices[train_size:]

def one_hot_encode(y, num_classes):
    return np.eye(num_classes)[y] 
X =iris.data
y = iris.target
#####
data = pd.DataFrame(X, columns=iris.feature_names)
data.insert(0, "bias padding", 1,)


data['label'] = y

# X = np.insert(X, 0, 1, axis=1)

X = np.insert(X, 0, 1, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def vectorized_result(j,nClasses):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a class
    (0...n) into a corresponding desired output from the neural
    network."""
    e = np.zeros((nClasses, 1))
    e[j] = 1.0
    return e

training_inputs = [np.reshape(x, (5, 1)) for x in X_train]
training_results = [vectorized_result(y,3) for y in y_train]
training_data = list(zip(training_inputs, training_results))
validation_inputs = [np.reshape(x, (5, 1)) for x in X_test]
validation_data = list(zip(validation_inputs, y_test))

model = Network([5, 3, 3])
model.SGD(training_data=training_data,epochs = 3000, mini_batch_size = 10, eta = 3, test_data=validation_data)



Epoch 0: 9 / 30
Epoch 1: 25 / 30
Epoch 2: 29 / 30
Epoch 3: 26 / 30
Epoch 4: 21 / 30
Epoch 5: 21 / 30
Epoch 6: 21 / 30
Epoch 7: 22 / 30
Epoch 8: 19 / 30
Epoch 9: 21 / 30
Epoch 10: 21 / 30
Epoch 11: 21 / 30
Epoch 12: 21 / 30
Epoch 13: 19 / 30
Epoch 14: 22 / 30
Epoch 15: 23 / 30
Epoch 16: 21 / 30
Epoch 17: 19 / 30
Epoch 18: 19 / 30
Epoch 19: 22 / 30
Epoch 20: 24 / 30
Epoch 21: 25 / 30
Epoch 22: 22 / 30
Epoch 23: 19 / 30
Epoch 24: 29 / 30
Epoch 25: 19 / 30
Epoch 26: 19 / 30
Epoch 27: 21 / 30
Epoch 28: 24 / 30
Epoch 29: 22 / 30
Epoch 30: 19 / 30
Epoch 31: 19 / 30
Epoch 32: 24 / 30
Epoch 33: 19 / 30
Epoch 34: 27 / 30
Epoch 35: 19 / 30
Epoch 36: 22 / 30
Epoch 37: 19 / 30
Epoch 38: 25 / 30
Epoch 39: 18 / 30
Epoch 40: 23 / 30
Epoch 41: 21 / 30
Epoch 42: 19 / 30
Epoch 43: 21 / 30
Epoch 44: 22 / 30
Epoch 45: 23 / 30
Epoch 46: 21 / 30
Epoch 47: 19 / 30
Epoch 48: 19 / 30
Epoch 49: 19 / 30
Epoch 50: 21 / 30
Epoch 51: 24 / 30
Epoch 52: 26 / 30
Epoch 53: 21 / 30
Epoch 54: 27 / 30
Epoch 55: 24 / 30
Epo